In [6]:
import typing
from typing import List, Dict, Tuple
import json


import pandas as pd
import gensim
from gensim.models import Word2Vec
from sklearn.cluster import DBSCAN
from numpy import dot, ndarray
import numpy as np

import matplotlib.pyplot as plt
from pandas.plotting import table

import nltk
from nltk.corpus import stopwords

# Download the stop words set
nltk.download('stopwords')


# import capturePPMI
from capturePPMI import Corpus , get_sorted_positive_words

import importlib
import capturePPMI
importlib.reload(capturePPMI)


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/chenyujie/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<module 'capturePPMI' from '/Users/chenyujie/Library/CloudStorage/OneDrive-QueenMary,UniversityofLondon/tensorflow-test/capturePPMI.py'>

07_whole year
08_whole year 
09_04 year
10_01
11_2

In [2]:
def reading_data(file_path:str , header: str = 'text',encoding: str = 'ISO-8859-1')->List[str]:
    print(file_path)
    df = pd.read_csv(file_path, header='infer',encoding=encoding)
    df = df.dropna(subset=[header])    # Here may have some column with nan-value, but this is wired, why here have column with nan value, I cannot understand??
    corpus = df[header].tolist()
    corpus = Corpus(corpus)
    return corpus.get_corpus()

In [3]:
file_paths = ["/Users/chenyujie/Desktop/ClassifiedData/Reddit/2007.csv",
              "/Users/chenyujie/Desktop/ClassifiedData/Reddit/2008.csv",
              "/Users/chenyujie/Desktop/ClassifiedData/Reddit/RC_2009-04_done.csv",
             "/Users/chenyujie/Desktop/ClassifiedData/Reddit/RC_2010-01_done.csv",
             "/Users/chenyujie/Desktop/ClassifiedData/Reddit/RC_2011-02_done.csv",
             "/Users/chenyujie/Desktop/ClassifiedData/Reddit/2014_02_done.csv"]

In [4]:
tokenized_corpus = [ reading_data(path) for path in file_paths]

/Users/chenyujie/Desktop/ClassifiedData/Reddit/2007.csv


Tokenizing: 100%|██████████| 8/8 [00:49<00:00,  6.13s/it]


/Users/chenyujie/Desktop/ClassifiedData/Reddit/2008.csv


Tokenizing: 100%|██████████| 8/8 [07:50<00:00, 58.79s/it] 


/Users/chenyujie/Desktop/ClassifiedData/Reddit/RC_2009-04_done.csv


...

Tokenizing: 100%|██████████| 8/8 [02:12<00:00, 16.60s/it] 


/Users/chenyujie/Desktop/ClassifiedData/Reddit/RC_2011-02_done.csv


Tokenizing: 100%|██████████| 8/8 [04:33<00:00, 34.19s/it] 


/Users/chenyujie/Desktop/ClassifiedData/Reddit/2014_02_done.csv


Tokenizing: 100%|██████████| 8/8 [03:22<00:00, 25.31s/it] 


In [44]:
for item in (tokenized_corpus):
    print (len(item))

24163
201332
27366
62653
125450
103431


In [7]:
# save the tokenized courpus


# # Example: Storing sorted tokenized words
# with open('/content/drive/MyDrive/TiwClassifcatedData/2014/2014_tokenized_sentences.json', 'w') as f:
#     json.dump(tokenized_corpus[0], f, indent=4)

def store_tokenized_sentences(tokenized_corpus: List[List[str]], file_info: List[str]):
    """
    Store tokenized sentences in JSON files named after their corresponding file information.

    Args:
        tokenized_corpus (List[List[str]]): List of tokenized sentences.
        file_info (List[str]): List of file information strings.

    Returns:
        None
    """
    if len(tokenized_corpus) != len(file_info):
        raise ValueError("Lengths of tokenized_corpus and file_info must be the same.")

    for i in range(len(tokenized_corpus)):
        file_path = f"./{file_info[i]}_tokenized_sentences.json"
        with open(file_path, 'w') as f:
            json.dump(tokenized_corpus[i], f, indent=4)


file_info = ['07','08','09','10','11','14']
store_tokenized_sentences(tokenized_corpus, file_info)



NameError: name 'tokenized_corpus' is not defined

In [8]:
# tokenized_corpus = [ reading_data(path) for path in file_paths]
# Do not need to tokenize once more
def load_tokenized_sentences(file_info: List[str]) -> List[List[str]]:
    """
    Load tokenized sentences from JSON files named after their corresponding file information.

    Args:
        file_info (List[str]): List of file information strings.

    Returns:
        List[List[str]]: List of tokenized sentences.
    """
    tokenized_corpus = []
    for info in file_info:
        file_path = f"Reddit_data_tokenized/{info}_tokenized_sentences.json"
        with open(file_path, 'r') as f:
            tokenized_sentence = json.load(f)
            tokenized_corpus.append(tokenized_sentence)
    return tokenized_corpus

file_info = ['07','08','09','10','11','14']
tokenized_corpus = load_tokenized_sentences(file_info)
print(len(tokenized_corpus))

6


In [9]:
model_07 = Word2Vec (sentences=tokenized_corpus[0], vector_size=200, window=10, min_count=30, workers=4)
model_08 = Word2Vec (sentences=tokenized_corpus[1], vector_size=200, window=10, min_count=30, workers=4)
model_09 = Word2Vec (sentences=tokenized_corpus[2], vector_size=200, window=10, min_count=30, workers=4)
model_10 = Word2Vec (sentences=tokenized_corpus[3], vector_size=200, window=10, min_count=30, workers=4)
model_11 = Word2Vec (sentences=tokenized_corpus[4], vector_size=200, window=10, min_count=30, workers=4)
model_14 = Word2Vec (sentences=tokenized_corpus[5], vector_size=200, window=10, min_count=30, workers=4)

# free memory
# for text in tokenized_corpus: del text

In [10]:
models = [ model_07,model_08,model_09, model_10, model_11, model_14]
# normalised all vectors
for model in models:
    model.wv.fill_norms(force=False)

# Defind the toixc word
## In my assumption, those word don not changing their semantic meaning

In [11]:
slut_words = ["nigger","ass","motherfucker","faggot"]   # This four words are absolute aggressive and toxic. Normally they are very dirty

In [12]:
for model in   models:
    print (model.wv.similarity('nigger', 'faggot'))


0.7931834
0.54882663
0.84654844
0.7353959
0.5928303
0.54599667


# They show high level similarity

In [13]:
# Get the mean vector using the KeyedVectors API
mean_vectors = [ model.wv.get_mean_vector(keys=slut_words) for model in models ]

# mean_vector = model_09.wv.get_mean_vector(keys=slut_words)
# print("Mean Vector:", mean_vector)

In [14]:
 # (model_09.wv.most_similar(positive = mean_vector))
for vector , model in   zip (mean_vectors,models ):
    print ('the most similar words: ',model.wv.most_similar(positive = vector))

the most similar words:  [('faggot', 0.9048381447792053), ('motherfucker', 0.8778119087219238), ('douche', 0.8724998831748962), ('fag', 0.8669649362564087), ('cock', 0.8596827387809753), ('smoke', 0.8561496734619141), ('sand', 0.854724645614624), ('nigger', 0.8499343991279602), ('lt', 0.8481256365776062), ('wanna', 0.8473266363143921)]
the most similar words:  [('faggot', 0.7510127425193787), ('motherfucker', 0.7449749112129211), ('nigger', 0.7172947525978088), ('nigga', 0.6101936101913452), ('cracker', 0.5904769897460938), ('cocksucker', 0.578001081943512), ('ass', 0.5658549666404724), ('whitey', 0.5502826571464539), ('bro', 0.5405650734901428), ('fag', 0.5401891469955444)]
the most similar words:  [('fuckin', 0.8803086280822754), ('u', 0.8794599771499634), ('faggot', 0.8788029551506042), ('nigger', 0.8722068667411804), ('motherfucker', 0.871536910533905), ('cunt', 0.8569853901863098), ('twat', 0.8193657994270325), ('cocksucker', 0.8161558508872986), ('pussy', 0.8128402233123779), ('s

# our mean of sult word make very sense!!!!

In [15]:
# load the list of toxic word that have high coefficient
shifting_words = get_sorted_positive_words('/Users/chenyujie/Library/CloudStorage/OneDrive-QueenMary,UniversityofLondon/tensorflow-test/coefs_with_fns_2023.pkl')

In [16]:
def remove_stop_words(words_list, extra_stop_words=None):
    """
    Remove stop words from a list of words.

    Args:
    words_list (list): List of words (strings).
    extra_stop_words (list): Additional stop words to consider.

    Returns:
    list: A list of words with stop words removed.
    """
    # Load default English stopwords
    stop_words = set(stopwords.words('english'))

    # Add any extra stop words provided
    if extra_stop_words:
        stop_words.update(extra_stop_words)

    # Filter out stop words from the list
    filtered_words = [word for word in words_list if word not in stop_words]

    return filtered_words

# Example usage
extra_words = ['be', 'of','I',"'s",'3','=','+','u','']  # Example of extra words you want to consider as stop words

# Remove stop words including any extra specified
shifting_words = remove_stop_words(shifting_words, extra_stop_words=extra_words)

print(shifting_words)


['𝟸𝟹𝟶𝟷𝟶𝟷', '𝟮𝟵𝟭', '𝟬𝟵𝟲𝟰𝟭𝟱𝟬𝟲𝟵𝟳𝟰', '𝟬𝟴𝟰', '𝟱𝟴𝟴𝟴', '𝟭𝟳𝟬', '𝟮𝟱𝟲', '𝗖𝟱𝟬', '𝟭𝟰𝟬𝘁𝗵', '𝟲𝟮𝟯𝟬', '𝟯𝟮', '𝟮𝟭𝟴', '𝟭𝟭𝟰𝟯𝟰', '𝟮𝟬𝟳', '𝟭𝟬𝟯𝟲𝟱', '𝟬𝟬𝗽𝗺', '𝟬𝟬', '𝟎𝟗𝟎𝟑𝟒𝟓𝟓𝟑𝟐𝟕𝟒', '𝟎𝟖𝟏𝟒𝟗𝟕𝟎𝟒𝟎𝟖𝟒', '𝟎𝟕𝟎𝟑𝟏𝟑𝟕𝟑𝟔𝟗𝟗', '𝟏𝟕', '𝟓𝟔𝟎', '𝟏𝟔', '𝟕𝟓𝟏', '𝟑𝟓𝐭𝐡', '𝟐𝟒𝐇', '𝟐𝟑𝟓', '𝟏𝟐𝟎', '𝟗𝟎𝟑𝟔𝟒𝟕', '𝟐𝟎𝟐𝟏', '𝟐𝟎𝐗', '𝟐𝟎𝐭𝐡', '𝟏𝟎𝟎𝐊', '𝙷𝚞𝚛𝚛𝚢', '𝚜𝚞𝚌𝚔', '𝙳𝚘𝚗', '𝙼𝚒𝚜𝚜', '𝚖𝚒𝚗𝚍', '𝚍𝚒𝚊𝚛𝚢', '𝚜𝚒𝚝', '𝙿𝚒𝚡𝚎𝚕𝚊𝚛𝚝', '𝙲𝚑𝚎𝚌𝚔𝚖𝚊𝚝𝚎', '𝚛𝚎𝚊𝚍', '𝙽𝚎𝚝𝚏𝚕𝚒𝚡', 'p𝚊treon', '𝙹𝚊𝚗𝚞𝚊𝚛𝚢', '𝚂𝚊𝚕𝚎', '𝚛𝚊𝚟𝚎𝚗𝚘𝚞𝚜', '𝚁𝚊𝚏𝚏𝚕𝚎', '𝚇𝚇𝚇', '𝙾𝚄𝚁', '𝙿𝙾𝚃𝚃𝙴𝚁', '𝚄𝙽𝙸𝚅𝙴𝚁𝚂𝙴𓍢', '𝙼𝙸𝙻𝙴', '𝙶𝙸𝚅𝙴', '𓄰𝙷𝙰𝚁𝚁𝚈', '𝚃𝙴𝙼𝙿𝚃𝙰𝚃𝙸𝙾𝙽', '𝙼𝙲𝙸𝙽𝚃𝙾𝚂𝙷', '𝙚𝙭𝙩𝙧𝙖𝙤𝙧𝙙𝙞𝙣𝙖𝙧𝙮', '𝙈𝙪𝙡𝙡𝙞𝙘𝙠', '𝙥𝙪𝙧𝙧', '𝙎𝙪𝙥𝙥𝙤𝙧𝙩', '𝙝𝙪𝙢𝙖𝙣', '𝙃𝙪𝙢𝙖𝙣𝙨', '𝙋𝙪𝙣𝙠𝙨', '𝙖𝙪𝙩𝙤𝙣𝙤𝙈𝙀', '𝙅𝙪𝙣𝙜𝙠𝙤𝙤𝙠', '𝙧𝙪𝙨𝙝', '𝙗𝙪𝙡𝙡𝙮', '𝙛𝙪𝙚𝙡', '𝙤𝙪𝙩𝙘𝙖𝙨𝙩𝙨', '𝘾𝙪𝙢', '𝙅𝙪𝙖𝙧𝙚𝙯', '𝙨𝙩𝙪𝙣𝙣𝙞𝙣𝙜', '𝙎𝙩𝙤𝙧𝙮', '𝙖𝙩𝙩𝙚𝙢𝙥𝙩', '𝙨𝙩𝙧𝙚𝙖𝙢𝙞𝙣𝙜', '𝙎𝙩𝙤𝙘𝙠', '𝘼𝙩𝙩𝙚𝙣𝙩𝙞𝙤𝙣', '𝙐𝙨𝙚𝙨', '𝘿𝙧𝙞𝙫𝙚', '𝙗𝙧𝙚𝙖𝙩𝙝𝙡𝙚𝙨𝙨', '𝙗𝙧𝙞𝙡𝙡𝙞𝙖𝙣𝙩', '𝙖𝙧𝙙𝙪𝙞𝙣𝙤𝙧', '𝙖𝙧𝙧𝙍', '𝘿𝙧𝙚𝙖𝙢𝙮', '𝙘𝙧𝙖𝙫𝙞𝙣𝙜𝙨', '𝙛𝙧𝙚𝙖𝙠𝙨', '𝘽𝙧𝙞𝙩𝙩𝙣𝙚𝙮', '𝙂𝙧𝙞𝙣𝙚𝙧', '𝙤𝙥𝙚𝙣', '𝙝𝙤𝙪𝙨𝙚', '𝙋𝙤𝙪𝙣𝙙', '𝙗𝙤𝙬𝙡', '𝙘𝙤𝙡𝙙', '𝘿𝙤𝙣𝙖𝙩𝙞𝙤𝙣', '𝙏𝙤𝙣𝙜', '𝙙𝙤𝙜𝙪𝙞𝙣𝙝𝙤𝙨', '𝙈𝙤𝙤𝙣𝘽𝙮𝙪𝙡', '𝙡𝙤𝙖𝙙𝙞𝙣𝙜', '𝘿𝙤𝙬𝙣𝙡𝙤𝙖𝙙', '𝙁𝙤𝙧𝙚𝙘𝙖𝙨𝙩', '𝙎𝙤𝙡𝙪𝙩𝙞𝙤𝙣', '𝙃𝙤𝙢𝙚', 

In [17]:
# remove all words do not appear in four models:
for model in models:
    words = model.wv.index_to_key
    # Filter out words from all_words that are not in keep_words
    shifting_words = [word for word in shifting_words if word in words]


In [18]:
print (len( shifting_words))

677


In [19]:
print (shifting_words)

['hypocritical', 'hypocrite', 'exist', 'ex', 'extra', 'exact', 'explain', 'except', 'existence', 'extremist', 'exactly', 'expose', 'twice', 'aware', 'swear', 'awful', 'owner', 'twat', 'evolution', 'evil', 'everybody', 'evidence', 'submit', 'fund', 'current', 'murder', 'due', 'building', 'guilty', 'must', 'guy', 'build', 'guarantee', 'nuke', 'bum', 'surely', 'ruin', 'yup', 'huh', 'cultural', 'bush', 'cure', 'european', 'rub', 'russia', 'buck', 'sure', 'multiple', 'supposedly', 'author', 'superior', 'quit', 'punch', 'cuz', 'culture', 'dump', 'justify', 'bunch', 'humanity', 'suck', 'dude', 'lunatic', 'sue', 'muslims', 'dumb', 'sucker', 'nut', 'butt', 'fucking', 'fuck', 'fucker', 'fuckin', 'pussy', 'dumbass', 'fucktard', 'cunt', 'nutjob', 'bullshit', 'status', 'state', 'start', 'gt', 'store', 'stone', 'stay', 'attempt', 'otherwise', 'stick', 'steal', 'stab', 'straight', 'street', 'ethnic', 'stop', 'stink', 'states', 'attract', 'wtf', 'stupid', 'stereotype', 'stfu', 'asian', 'us', 'use', 'i

In [20]:
# Write the list to a text file, one word per line
with open('shifting_words.txt', 'w') as file:
    for word in shifting_words:
        file.write(word + '\n')

In [21]:
# get all word , find similiaty and gradinet

In [22]:
def find_word_vectors(target_words: List[str], models: List[Word2Vec] ) -> Dict[str, Dict[str, np.ndarray]]:
    """
    Retrieve the word vectors for a list of target words across multiple Word2Vec models.

    Args:
        target_words (List[str]): List of words for which vectors are required.
        models (List[Word2Vec]): List of Word2Vec models from which vectors are to be retrieved.

    Returns:
        Dict[str, Dict[str, np.ndarray]]: A dictionary where keys are words and values are dictionaries.
            Each nested dictionary has model identifiers as keys and the corresponding word vectors as values.
    """
    words_similarities = {}
    # Iterate through each target word
    for word in target_words:
        model_vectors = [None]* len(models)
        # Retrieve vectors from each model
        for i, model in enumerate( models):
            try:
                # Get vector for the word from the current model
                vector = model.wv[word]
                model_vectors[i] = vector
            except KeyError:
                # Handle the case where the word is not in the model's vocabulary
                model_vectors[model] = None
        # Store vectors from all models for the current word
        words_similarities[word] = model_vectors

    return words_similarities

In [23]:
from capturePPMI import capture_cosine_similarity

def find_sim(words_vectors: Dict[str, List[np.ndarray]], words: List[str], centre_word_vector: List[np.ndarray]) -> Dict[str, List[float]]:
    """
    Find the similarity of vectors of words to corresponding center word vectors.

    Args:
    words_vectors (Dict[str, List[np.ndarray]]): Dictionary mapping words to lists of their vectors.
    words (List[str]): List of words to find similarities for.
    centre_word_vector (List[np.ndarray]): List of central word vectors, one for each word.

    Returns:
    Dict[str, List[float]]: Dictionary mapping words to their similarity scores with the center word vectors.
    """
    similarity_scores = {}
    for word in words:
        if word in words_vectors:
            vectors = words_vectors[word]
            word_similarity = [capture_cosine_similarity(vector, centre_word_vector[i]) for i, vector in enumerate(vectors)]
            similarity_scores[word] = word_similarity
        else:
            # Handling cases where the word might not be in the dictionary
            similarity_scores[word] = f'Word {word} not in dictionary'

    return similarity_scores

In [24]:
def calculate_similarity_gradient(similarities: List[float]) -> List[float]:
    """
    Calculate the gradient of similarity scores between consecutive elements.

    Args:
    similarities (List[float]): List of similarity scores for a word across different models or times.

    Returns:
    List[float]: List of gradients between consecutive similarity scores.
    """
    # Calculate the gradients between consecutive similarity scores
    gradients = [similarities[i] - similarities[i - 1] for i in range(1, len(similarities))]
    return gradients

In [25]:
def categorize_gradient_words(gradient_data: Dict[str, List[float]]) -> Tuple[List[str], List[str]]:
    """
    Categorize words based on their similarity score gradients into increasing or decreasing lists.

    Args:
    gradient_data (Dict[str, List[float]]): Dictionary where each key is a word and the value is a list of gradients of similarity scores.

    Returns:
    Tuple[List[str], List[str]]: A tuple containing two lists:
        - The first list contains words with increasing gradients.
        - The second list contains words with decreasing gradients.
    """
    increasing_gradients = []
    decreasing_gradients = []

    for word, gradients in gradient_data.items():
        if all(g > 0 for g in gradients):  # Check if all gradients are positive
            increasing_gradients.append(word)
        elif all(g < 0 for g in gradients):  # Check if all gradients are negative
            decreasing_gradients.append(word)

    return increasing_gradients, decreasing_gradients


In [26]:
shifting_words_vector = find_word_vectors(shifting_words,  models)

In [27]:
shifting_words_vector_sim = find_sim (shifting_words_vector, shifting_words,mean_vectors )

In [28]:
print (shifting_words_vector_sim['ass'])

[0.7701645, 0.5658549, 0.74263495, 0.6575536, 0.5601168, 0.56674856]


In [29]:
gradient_results = {}
for word, scores in shifting_words_vector_sim.items():
    gradient_results[word] = calculate_similarity_gradient(scores)
    print (gradient_results[word])
print(gradient_results)

[-0.3956371, 0.47983634, -0.3215436, -0.21128011, 0.09120887]
[-0.41924128, 0.09019196, 0.03631085, -0.17131492, -0.000331223]
[-0.09062939, 0.13898416, -0.13481809, -0.026334167, -0.048488945]
[-0.5584588, 0.44699392, -0.45299456, -0.00090857595, -0.036652632]
[-0.33556604, 0.03855937, -0.13403678, 0.06744859, -0.25850147]
[-0.29496732, 0.24786183, -0.15904923, -0.15216428, 0.15183836]
[-0.4051359, 0.35552755, -0.29596388, -0.016568147, -0.02749234]
[-0.057947107, -0.1383212, 0.19376487, -0.034879535, -0.06873231]
[-0.42177555, 0.17741299, -0.05979067, -0.12488961, 0.10680981]
[0.021596216, -0.12069382, 0.116871774, 0.043433063, -0.04043138]
[-0.2036171, 0.19939497, -0.14943585, -0.020420507, -0.0036658794]
[-0.3887369, 0.42413092, -0.23356217, -0.2551785, 0.1482476]
[-0.17081895, 0.1933397, -0.24429011, 0.035051167, -0.105913594]
[-0.34449324, 0.14807826, -0.15874077, 0.006563902, 0.14994274]
[-0.5609043, 0.35571396, -0.3078092, -0.1401749, 0.13166437]
[-0.63341266, 0.5395146, -0.427

In [30]:
increasing_words, decreasing_words = categorize_gradient_words(gradient_results)


In [31]:
def categorize_and_sort_gradient_words(gradient_data: Dict[str, List[float]]) -> Tuple[List[str], List[str]]:
    """
    Categorize and sort words based on their similarity score gradients into increasing or decreasing lists,
    sorted by the mean of their gradients.

    Args:
    gradient_data (Dict[str, List[float]]): Dictionary where each key is a word and the value is a list of gradients of similarity scores.

    Returns:
    Tuple[List[str], List[str]]: A tuple containing two sorted lists:
        - The first list contains words with increasing gradients sorted by the mean of gradients.
        - The second list contains words with decreasing gradients sorted by the mean of gradients.
    """
    increasing_gradients = []
    decreasing_gradients = []

    for word, gradients in gradient_data.items():
        if all(g > 0 for g in gradients):  # Check if all gradients are positive
            increasing_gradients.append((word, sum(gradients) / len(gradients)))
        elif all(g < 0 for g in gradients):  # Check if all gradients are negative
            decreasing_gradients.append((word, sum(gradients) / len(gradients)))

    # Sort the tuples by the mean of the gradients and extract the word for final list
    sorted_increasing_words = [word for word, _ in sorted(increasing_gradients, key=lambda x: x[1], reverse=True)]
    sorted_decreasing_words = [word for word, _ in sorted(decreasing_gradients, key=lambda x: x[1])]

    return sorted_increasing_words, sorted_decreasing_words

# Example usage
sorted_increasing_words, sorted_decreasing_words = categorize_and_sort_gradient_words(gradient_results)

# print("Sorted Increasing Words by Gradient Mean:", sorted_increasing_words)
# print("Sorted Decreasing Words by Gradient Mean:", sorted_decreasing_words)


In [32]:
for word in increasing_words:
    print (f'the word {word} is increasing similarity with sim {shifting_words_vector_sim[word]} \ngradient {gradient_results[word]}')

the word laden is increasing similarity with sim [-0.18506154, -0.097521566, -0.02476253, 0.022641357, 0.06851546, 0.11198921] 
gradient [0.08753998, 0.07275903, 0.047403887, 0.0458741, 0.04347375]


In [33]:
for word in decreasing_words:
    print (f'the word {word} is decreasing similarity with sim {shifting_words_vector_sim[word]} \ngradient {gradient_results[word]}')

the word man is decreasing similarity with sim [0.1851226, 0.18429771, 0.16727324, 0.09243343, 0.03691371, 0.032039627] 
gradient [-0.0008248836, -0.017024472, -0.07483981, -0.05551972, -0.004874084]


In [34]:
def sort_by_last_similarity_score(similarity_scores):
    """
    Sort words by the last value in their similarity scores list.

    Args:
    similarity_scores (Dict[str, List[float]]): Dictionary of words and their list of similarity scores.

    Returns:
    List[str]: List of words sorted by the last value in their similarity scores.
    """
    # Filter out words with no similarity score or placeholder text
    filtered_scores = {word: scores for word, scores in similarity_scores.items() if isinstance(scores, list) and scores}
    # Sort words based on the last similarity score
    sorted_words = sorted(filtered_scores, key=lambda x: filtered_scores[x][-1], reverse=True)
    return sorted_words

def sort_by_first_similarity_score(similarity_scores):
    """
    Sort words by the first value in their similarity scores list.

    Args:
    similarity_scores (Dict[str, List[float]]): Dictionary of words and their list of similarity scores.

    Returns:
    List[str]: List of words sorted by the first value in their similarity scores.
    """
    # Filter out words with no similarity score or placeholder text
    filtered_scores = {word: scores for word, scores in similarity_scores.items() if isinstance(scores, list) and scores}
    # Sort words based on the first similarity score
    sorted_words = sorted(filtered_scores, key=lambda x: filtered_scores[x][0], reverse=True)
    return sorted_words




In [35]:
# Sort the words
words_sorted_by_last = sort_by_last_similarity_score(shifting_words_vector_sim)
words_sorted_by_first = sort_by_first_similarity_score(shifting_words_vector_sim)

print("Words sorted by last similarity score:", words_sorted_by_last[:20])
print("Words sorted by first similarity score:", words_sorted_by_first[:20])


Words sorted by last similarity score: ['motherfucker', 'faggot', 'cracker', 'nigger', 'cunt', 'fag', 'twat', 'dumbass', 'prick', 'fuckin', 'ass', 'mr', 'tard', 'douche', 'bastard', 'bitch', 'dude', 'loser', 'douchebag', 'asshat']
Words sorted by first similarity score: ['faggot', 'motherfucker', 'douche', 'fag', 'cock', 'smoke', 'nigger', 'cunt', 'whiny', 'smell', 'dick', 'troll', 'prick', 'douchebag', 'goddamn', 'pussy', 'brain', 'ya', 'mouth', 'swear']


In [36]:
def sort_words_by_last_gradient(gradient_data: Dict[str, List[float]]) -> List[str]:
    """
    Sort words based on the last gradient value in their list of gradients.

    Args:
    gradient_data (Dict[str, List[float]]): Dictionary of words and their list of gradient values.

    Returns:
    List[str]: List of words sorted by the last gradient value, descending.
    """
    # Filter to ignore empty lists or non-list items
    filtered_data = {word: gradients for word, gradients in gradient_data.items() if isinstance(gradients, list) and gradients}
    # Sort words based on the last gradient value
    sorted_words = sorted(filtered_data, key=lambda x: filtered_data[x][-1], reverse=True)
    return sorted_words

def sort_words_by_average_gradient(gradient_data: Dict[str, List[float]]) -> List[str]:
    """
    Sort words based on the average of their gradient values.

    Args:
    gradient_data (Dict[str, List[float]]): Dictionary of words and their list of gradient values.

    Returns:
    List[str]: List of words sorted by the average of their gradient values, descending.
    """
    # Filter to ignore empty lists or non-list items
    filtered_data = {word: gradients for word, gradients in gradient_data.items() if isinstance(gradients, list) and gradients}
    # Sort words based on the average gradient value
    sorted_words = sorted(filtered_data, key=lambda x: filtered_data[x][0], reverse=True)
    return sorted_words

def sort_words_by_sum_gradient(gradient_data: Dict[str, List[float]], sim: Dict[str, List[float]], min_threshold: float = None, max_threshold: float = None) -> List[str]:
    """
    Sort words based on the sum of their gradient values, excluding words that do not meet the specified similarity thresholds.

    Args:
    gradient_data (Dict[str, List[float]]): Dictionary of words and their list of gradient values.
    sim (Dict[str, List[float]]): Dictionary of words and their list of similarity scores.
    min_threshold (float, optional): Minimum similarity score threshold to include a word in the sorting.
    max_threshold (float, optional): Maximum similarity score threshold to include a word in the sorting.

    Returns:
    List[str]: List of words sorted by the sum of their gradient values, descending, that meet the similarity score thresholds.
    """
    # Filter to ignore empty lists, non-list items, and filter by similarity thresholds
    filtered_data = {word: gradients for word, gradients in gradient_data.items()
                     if isinstance(gradients, list) and gradients and
                        (min_threshold is None or max(sim.get(word, [0])) >= min_threshold) and
                        (max_threshold is None or max(sim.get(word, [0])) <= max_threshold)}

    # Sort words based on the sum of the gradient values
    sorted_words = sorted(filtered_data, key=lambda x: sum(filtered_data[x]), reverse=True)
    return sorted_words




words_sorted_by_last = sort_words_by_last_gradient(gradient_results)
words_sorted_by_average = sort_words_by_average_gradient(gradient_results)



# getting the toxic words are not same as toxic word

# Let the max threshold be the min similiary of slut word 
# the min can be any thinng 

In [37]:
for word in slut_words:
    print ("word",word ,gradient_results[word])
    print (shifting_words_vector_sim[word])

word nigger [-0.13263965, 0.15491217, -0.004148066, -0.1473239, -0.014590323]
[0.84993434, 0.7172947, 0.87220687, 0.8680588, 0.7207349, 0.7061446]
word ass [-0.20430958, 0.17678005, -0.08508134, -0.097436786, 0.006631732]
[0.7701645, 0.5658549, 0.74263495, 0.6575536, 0.5601168, 0.56674856]
word motherfucker [-0.13283694, 0.126562, -0.026494622, -0.09044194, 0.016976535]
[0.87781185, 0.7449749, 0.8715369, 0.8450423, 0.75460035, 0.7715769]
word faggot [-0.1538254, 0.12779039, -0.03775543, -0.059142113, -0.04506868]
[0.90483814, 0.75101274, 0.87880313, 0.8410477, 0.7819056, 0.7368369]


In [55]:
min_threshold_value = -1 # Define a minimum threshold for maximum similarity
max_threshold_value = 0.5 # 0.55 # Define a maximum threshold for maximum similarity


words_sorted_by_sum = sort_words_by_sum_gradient(gradient_results, shifting_words_vector_sim, min_threshold=min_threshold_value, max_threshold=max_threshold_value)

print (len(words_sorted_by_sum))


for word in words_sorted_by_sum[:20]:
    print ("word",word,gradient_results[word])
    print (shifting_words_vector_sim[word])



print("\n The reduce similiary")


for word in words_sorted_by_sum[-20:]:
    print ("word",word ,gradient_results[word])
    print (shifting_words_vector_sim[word])


437
word state [0.41641492, -0.26658282, 0.14373721, 0.21548194, -0.12161724]
[-0.4693304, -0.052915495, -0.3194983, -0.17576109, 0.039720852, -0.08189639]
word mexican [0.5282335, -0.18712568, 0.048160676, 0.13172272, -0.14358336]
[-0.28936002, 0.23887353, 0.051747847, 0.09990852, 0.23163123, 0.088047884]
word bin [0.10979609, 0.106563315, -0.14863938, 0.21380544, 0.08643328]
[-0.15843658, -0.04864049, 0.057922825, -0.09071655, 0.123088896, 0.20952217]
word nation [0.31946278, -0.2341919, 0.11291692, 0.15515737, -0.0029708543]
[-0.3460458, -0.026583018, -0.2607749, -0.147858, 0.007299378, 0.0043285238]
word usa [0.20875092, -0.11442639, 0.047160648, 0.106839284, 0.0900514]
[-0.24666844, -0.03791753, -0.15234391, -0.105183266, 0.0016560219, 0.09170742]
word country [0.26782927, -0.13163751, 0.11886833, 0.039356127, 0.037111536]
[-0.18788941, 0.07993985, -0.051697664, 0.067170665, 0.10652679, 0.14363833]
word communist [0.15568152, 0.097351745, -0.13887456, 0.11547603, 0.09804112]
[-0.2

In [56]:
def categorize_words_by_gradient_sign( words:List[str] ,gradient_data: Dict[str, List[float]] ) -> (List[str], List[str]):
    """
    Categorize words into two lists based on their gradient signs: all positive or all negative.

    Args:
    gradient_data (Dict[str, List[float]]): Dictionary of words and their list of gradient values.

    Returns:
    Tuple[List[str], List[str]]:
        - First list contains words that have all positive gradient values.
        - Second list contains words that have all negative gradient values.
    """
    all_positive_words = []
    all_negative_words = []

    for word, gradients in gradient_data.items():
        if word not in words:
            continue
        if all(g > 0 for g in gradients):
            all_positive_words.append(word)
        elif all(g < 0 for g in gradients):
            all_negative_words.append(word)

    return all_positive_words, all_negative_words


all_positive, all_negative = categorize_words_by_gradient_sign(words_sorted_by_sum , gradient_results)
print("Words with all positive gradients:", len(all_positive))
for word in all_positive:
    print (word, "\n", gradient_results[word],"\n",shifting_words_vector_sim[word])
print("\nWords with all negative gradients:",len(all_negative))
for word in all_negative:

    print (word, "\n", gradient_results[word],"\n",shifting_words_vector_sim[word])


Words with all positive gradients: 1
laden 
 [0.08753998, 0.07275903, 0.047403887, 0.0458741, 0.04347375] 
 [-0.18506154, -0.097521566, -0.02476253, 0.022641357, 0.06851546, 0.11198921]

Words with all negative gradients: 1
man 
 [-0.0008248836, -0.017024472, -0.07483981, -0.05551972, -0.004874084] 
 [0.1851226, 0.18429771, 0.16727324, 0.09243343, 0.03691371, 0.032039627]


In [51]:
from prettytable import PrettyTable

# Function to print results in table format
def print_results(title, words, gradient_results, shifting_words_vector_sim):
    print(title)
    table = PrettyTable()
    table.field_names = ["Word","Similarity Scores"]

    for word in words:
        gradients = gradient_results[word]
        similarities = shifting_words_vector_sim[word]
        table.add_row([word, similarities])

    print(table)

# Example usage:
# Assuming all_positive, all_negative, gradient_results, and shifting_words_vector_sim are defined

print_results("Words with  positive gradients:", all_positive, gradient_results, shifting_words_vector_sim)
print("\nWords with all negative gradients:", len(all_negative))
print_results("Words with  negative gradients:", all_negative, gradient_results, shifting_words_vector_sim)


Words with  positive gradients:
+-------+-------------------------------------------------------------------------------+
|  Word |                               Similarity Scores                               |
+-------+-------------------------------------------------------------------------------+
| laden | [-0.18506154, -0.097521566, -0.02476253, 0.022641357, 0.06851546, 0.11198921] |
+-------+-------------------------------------------------------------------------------+

Words with all negative gradients: 1
Words with  negative gradients:
+------+--------------------------------------------------------------------------+
| Word |                            Similarity Scores                             |
+------+--------------------------------------------------------------------------+
| man  | [0.1851226, 0.18429771, 0.16727324, 0.09243343, 0.03691371, 0.032039627] |
+------+--------------------------------------------------------------------------+


In [57]:
all_positive

['laden']

In [58]:
all_negative

['man']

In [67]:
GloVe_word = ['bush', 'throat', 'barely', 'hate']
print_results("In word2vec embedding with words from  GloVE", GloVe_word, gradient_results, shifting_words_vector_sim)

In word2vec embedding with words from  GloVE
+--------+------------------------------------------------------------------------------+
|  Word  |                              Similarity Scores                               |
+--------+------------------------------------------------------------------------------+
|  bush  |  [0.087995954, 0.033594504, 0.03615771, 0.17933333, 0.1556255, 0.20111173]   |
| throat |    [0.5997046, 0.30237865, 0.64624184, 0.40228474, 0.3164845, 0.32796195]    |
| barely | [0.4604843, -0.14342342, 0.26529646, -0.10478931, -0.16654557, -0.13125926]  |
|  hate  | [0.20213673, 0.105792135, 0.23581222, 0.093999475, 0.009786381, 0.003267256] |
+--------+------------------------------------------------------------------------------+


In [68]:
PPMI_word =  ['start', 'grab', 'horrible', 'yo', 'park', 'pant','bush', 'americans', 'sheeple', 'fact']
print_results("In word2ve embedding with words from PPMI :", PPMI_word, gradient_results, shifting_words_vector_sim)


In word2ve embedding with words from PPMI :
+-----------+-----------------------------------------------------------------------------------+
|    Word   |                                 Similarity Scores                                 |
+-----------+-----------------------------------------------------------------------------------+
|   start   |   [0.038224243, -0.03160224, 0.18554068, 0.010907144, -0.008034899, 0.009719556]  |
|    grab   |     [0.48668197, 0.18226041, 0.48215133, 0.010335837, 0.09607863, 0.11403305]     |
|  horrible |    [0.18667242, -0.1833415, 0.07696259, -0.061457325, -0.19954643, -0.24405792]   |
|     yo    |       [0.7397184, 0.3881343, 0.7172638, 0.52750033, 0.34520096, 0.37989125]       |
|    park   |      [0.45913088, 0.14313757, 0.37631023, 0.05042863, 0.098679796, 0.0921022]     |
|    pant   |      [0.70398146, 0.36392754, 0.5383839, 0.27666873, 0.24171644, 0.19969916]      |
|    bush   |     [0.087995954, 0.033594504, 0.03615771, 0.17933333, 0.155

# All words

In [54]:
def print_results_withGrid(title, words, gradient_results, shifting_words_vector_sim):
    print(title)
    table = PrettyTable()
    table.field_names = ["Word","Similarity Scores","gradient"]

    for word in words:
        gradients = gradient_results[word]
        similarities = shifting_words_vector_sim[word]
        table.add_row([word, similarities,gradients])

    print(table)
print_results_withGrid("All words in Word2vec", shifting_words, gradient_results, shifting_words_vector_sim)


All words in Word2vec
+---------------+--------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|      Word     |                                  Similarity Scores                                   |                                gradient                                |
+---------------+--------------------------------------------------------------------------------------+------------------------------------------------------------------------+
|  hypocritical |        [0.5321492, 0.13651212, 0.61634845, 0.29480484, 0.08352473, 0.1747336]        |     [-0.3956371, 0.47983634, -0.3215436, -0.21128011, 0.09120887]      |
|   hypocrite   |         [0.6888643, 0.269623, 0.35981497, 0.39612582, 0.2248109, 0.22447968]         |    [-0.41924128, 0.09019196, 0.03631085, -0.17131492, -0.000331223]    |
|     exist     |   [-0.05274179, -0.14337118, -0.0043870243, -0.13920511, -0.16553928, 

In [105]:
'r'>'p'

True